In [1]:
#Iportacion de las librerias necesarias
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

2025-12-23 20:13:46.202166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 20:13:46.314556: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-23 20:13:48.924922: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/juan-jose/tf-gpu1/lib/python3.12/site-packages/keras/src/expor

In [2]:
#Importacion del dataset de ropa
dataset = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()
classNames = ['Camiseta/top', 'Pantalones', 'Sueter', 'Vestido', 'Abrigo', 'Sandalia', 'Camisa', 'Zapatilla deportiva', 'Bolso', 'Botas']

In [3]:
#preprocesamiento de los datos
train_images = train_images / 255.0
test_images = test_images / 255.0


In [4]:
#COnfiguracoin de las capas del modelo
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

/home/juan-jose/tf-gpu1/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1766538832.590281    6228 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1205 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
#Compilacion del modelo
model.compile(optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [6]:
#Entrenamiento del modelo
model.fit(train_images, train_labels, epochs=15
)

Epoch 1/15


2025-12-23 20:14:03.635982: I external/local_xla/xla/service/service.cc:163] XLA service 0x7944940047e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-23 20:14:03.636016: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-12-23 20:14:03.679140: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-23 20:14:03.879272: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
2025-12-23 20:14:03.914122: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-23 20:14:04.

  32/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.3830 - loss: 1.7892

I0000 00:00:1766538848.394083    6344 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8220 - loss: 0.5059
Epoch 2/15
1595/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8615 - loss: 0.3913

KeyboardInterrupt: 

In [8]:
#Evaluando el modelo 
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 3s - 9ms/step - accuracy: 0.8841 - loss: 0.3488

Test accuracy: 0.8841000199317932


In [9]:
#Adicion de capa para pasar las salidas lineales a probabilidades
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
                                           

In [50]:

def plotImage(i, arregloPredicciones, salidaVerdadera, img):
#Funcion para trazar la imagen   
    salidaVerdadera, img = salidaVerdadera[i], img[i]
    plt.grid(False)
    plt.yticks([])
    plt.xticks([])
    plt.imshow(img, cmap=plt.cm.binary)
    prediccionModelo = np.argmax(arregloPredicciones)#toma del arreglo de predicciones del modelo, el que tenga mas alto\

    if prediccionModelo == salidaVerdadera:
        color = 'green'
    else:
        color = 'red'
    plt.xlabel("{} {:2.0f}% ({})".format(classNames[prediccionModelo], 
                                         100*np.max(arregloPredicciones),
                                         classNames[salidaVerdadera]),
                                         color=color) 

In [51]:
def plotValueArray(i, arregloPredicciones, salidaVerdadera):
    trueLaber= salidaVerdadera[i]
    plt.grid(False)
    plt.yticks([])
    plt.xticks(range(10))   
    thisplot = plt.bar(range(10), arregloPredicciones, color="#777777") 
    plt.ylim([0, 1])
    predictedLabel = np.argmax(arregloPredicciones)

    thisplot[predictedLabel].set_color('red')
    thisplot[trueLaber].set_color('green')
    

In [79]:
def predecirUnaImagen(imagenIndex, test_labels, test_images):
    #Uso del modelo entrenado para predecir una imagen
    img = test_images[imagenIndex] #Seleccion de la imagen a predecir 
    #Adicion de la imagen a una lista de salida
    img = (np.expand_dims(img,0))
    predicciones = probability_model.predict(img)
    #Muestra la grafica de prección y siu exactitud en el grafico de barras 
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plotImage(imagenIndex, predicciones[0], test_labels, test_images)
    plt.subplot(1,2,2)
    plotValueArray(imagenIndex, predicciones[0], test_labels)
    plt.show()
  

In [1]:
predecirUnaImagen(2424, test_labels, test_images)

NameError: name 'predecirUnaImagen' is not defined